In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!nvidia-smi

Mon Oct  2 04:06:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

1. Install The hugging face transformers library

In [6]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00


2. Import the required libraries

In [7]:
import os
import torch
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset


4. Load the model and processor


In [8]:
# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-base")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")
model.config.forced_decoder_ids = None

6. Let's try to use an external audio in wav format

In [21]:
import librosa

# Specify the file path of the uploaded audio file
audio_path = "/content/drive/MyDrive/Colab Notebooks/Audio Clips/F_0101_10y4m_1.wav"  # Replace with the actual file name

# Load and display the audio
audio, sampling_rate = librosa.load(audio_path, sr=16000)
display(audio)

array([-0.00707609, -0.01186869, -0.01081028, ..., -0.00524877,
       -0.00460625, -0.00393208], dtype=float32)

In [30]:
# Load and convert audio to the format expected by Whisper
def preprocess_audio(audio_file_path):
    waveform, sample_rate = torchaudio.load(audio_file_path)
    if sample_rate != 16000:
        # Resample audio to 16kHz if not already
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)
    return waveform, 16000

#Reshape: Verify the shape of the audio_processed tensor. It should have dimensions [sequence_length] if it's mono audio or [2, sequence_length] if it's stereo. You can reshape it accordingly:

audio_processed, sampling_rate = preprocess_audio(audio_path)
if len(audio_processed.shape) == 2:
    audio_processed = audio_processed.mean(dim=0)  # If stereo, take the mean




In [31]:
input_features = processor(audio_processed, sampling_rate=sampling_rate, return_tensors="pt").input_features

In [32]:
predicted_ids = model.generate(input_features)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print(transcription)

['<|0.00|> What happened? What happened after she got out of the orphanage?<|6.00|><|8.00|> She found the old building where she used to live.<|12.00|><|20.00|> And she went to sightseeing.<|24.00|><|26.00|>']
